In [1]:
# Imports (some imports may not be necessary)
import numpy as np
import math
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, assemble, transpile
from qiskit.visualization import plot_histogram
import qiskit.quantum_info as qi

# Functions to define QFT gate in Qiskit
def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(math.pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)
    
def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qftCircuit(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [2]:
# 1 qubit QFT
n = 1

qc1 = QuantumCircuit(n,n)
qftCircuit(qc1,n)
qc1.draw()

┌───┐
q_0: ┤ H ├
     └───┘
c: 1/═════

In [3]:
state1 = qi.Statevector.from_instruction(qc1)
stateVec1 = state1.__array__()
print(stateVec1)

[0.70710678+0.j 0.70710678+0.j]


In [4]:
# Comment out these lines
import sys
sys.path.insert(0, 'C:\\Users\\masch\\Quantum Computing\\QComp\\pgmpy')

# Imports
import cmath
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.inference import BeliefPropagation

#QFT gate modeled using pgmpy

N = 2
omega_N = cmath.exp(2*math.pi*1j/N)
qft1 = BayesianNetwork([('q0m0','q0m1')])
cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[1], [0]])
cpd_q0m1 = TabularCPD(variable='q0m1', variable_card = 2, values = [[1/np.sqrt(N),1/np.sqrt(N)], [1/np.sqrt(N),omega_N/np.sqrt(N)]], evidence = ['q0m0'], evidence_card = [2])

"""
U_QFT =
[1/sqrt(2)   1/sqrt(2)]
[1/sqrt(2)   -1/sqrt(2)]
"""
qft1.add_cpds(cpd_q0m0,cpd_q0m1)
qftInfer1 = VariableElimination(qft1)
q1 = qftInfer1.query(['q0m1'])
print(q1)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

+---------+----------------+
| q0m1    |      phi(q0m1) |
+=========+================+
| q0m1(0) | 0.7071+0.0000j |
+---------+----------------+
| q0m1(1) | 0.7071+0.0000j |
+---------+----------------+


C:\Users\masch\Quantum Computing\QComp\pgmpy\pgmpy\extern\tabulate.py:282: ComplexWarning: Casting complex values to real discards the imaginary part
  n = conv(string)


In [5]:
# Obtain the ordering of the variables in the display above, as well as their values
q1Vars = q1.variables
q1Values = q1.values

print(q1Vars)
print(q1Values)

['q0m1']
[0.70710678+0.j 0.70710678+0.j]


In [6]:
# 2 qubit QFT

n = 2

qc2 = QuantumCircuit(n,n)
qftCircuit(qc2,n)
qc2.draw()

┌───┐   
q_0: ──────■───────┤ H ├─X─
     ┌───┐ │P(π/2) └───┘ │ 
q_1: ┤ H ├─■─────────────X─
     └───┘                 
c: 2/══════════════════════

In [7]:
state2 = qi.Statevector.from_instruction(qc2)
stateVec2 = state2.__array__()
print(stateVec2)

[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]


In [8]:
#QFT with 2 qubits

N = 4
omega_N = cmath.exp(2*math.pi*1j/N)
qft2 = BayesianNetwork([('q0m0','q0m1'), ('q1m0','q0m1'), ('q0m0','q1m1'), ('q1m0','q1m1')])
cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[1], [0]])
cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[1], [0]])
cpd_q0m1 = TabularCPD(variable='q0m1', variable_card = 2, values = [[1/np.sqrt(2),1/np.sqrt(2),1/np.sqrt(2),1/np.sqrt(2)], [1/np.sqrt(2),(omega_N**2)/np.sqrt(2),(omega_N**4)/np.sqrt(2),(omega_N**6)/np.sqrt(2)]], evidence = ['q0m0','q1m0'], evidence_card = [2,2])
cpd_q1m1 = TabularCPD(variable='q1m1', variable_card = 2, values = [[1/np.sqrt(2),1/np.sqrt(2),1/np.sqrt(2),1/np.sqrt(2)], [1/np.sqrt(2),(omega_N**1)/np.sqrt(2),(omega_N**2)/np.sqrt(2),(omega_N**3)/np.sqrt(2)]], evidence = ['q0m0','q1m0'], evidence_card = [2,2])

"""
U_QFT =
[1/2   1/2   1/2   1/2]
[1/2   i/2   -1/2   -i/2]
[1/2   -1/2   1/2   -1/2]
[1/2   -i/2   -1/2   i/2]
"""


qft2.add_cpds(cpd_q0m0,cpd_q0m1,cpd_q1m0,cpd_q1m1)
qftInfer2 = VariableElimination(qft2)
q2 = qftInfer2.query(['q0m1','q1m1'])
print(q2)

"""
U_QFT(00) = 1/2 00 + 1/2 01 + 1/2 10 + 1/2 11 = (1/sqrt(2)* (0 + 1)) * (1/sqrt(2)* (0 + 1))
U_QFT(01) = 1/2 00 + i/2 01 - 1/2 10 - i/2 11 = (1/sqrt(2)* (0 - 1)) * (1/sqrt(2)* (0 + i*1))
"""

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+---------+---------+------------------+
| q0m1    | q1m1    |   phi(q0m1,q1m1) |
+=========+=========+==================+
| q0m1(0) | q1m1(0) |   0.5000+0.0000j |
+---------+---------+------------------+
| q0m1(0) | q1m1(1) |   0.5000+0.0000j |
+---------+---------+------------------+
| q0m1(1) | q1m1(0) |   0.5000+0.0000j |
+---------+---------+------------------+
| q0m1(1) | q1m1(1) |   0.5000+0.0000j |
+---------+---------+------------------+


'\nU_QFT(00) = 1/2 00 + 1/2 01 + 1/2 10 + 1/2 11 = (1/sqrt(2)* (0 + 1)) * (1/sqrt(2)* (0 + 1))\nU_QFT(01) = 1/2 00 + i/2 01 - 1/2 10 - i/2 11 = (1/sqrt(2)* (0 - 1)) * (1/sqrt(2)* (0 + i*1))\n'

In [9]:
# Obtain the ordering of the variables in the display above, as well as their values
q2Vars = q2.variables
q2Values = q2.values

print(q2Vars)
print(q2Values)

['q0m1', 'q1m1']
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]


In [10]:
# 3 qubit QFT

n = 3

qc3 = QuantumCircuit(n,n)
qftCircuit(qc3,n)
qc3.draw()

┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────■───────┤ H ├─■─────────────┼─
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │ 
q_2: ┤ H ├─■────────■───────────────────────────X─
     └───┘                                        
c: 3/═════════════════════════════════════════════

In [11]:
state3 = qi.Statevector.from_instruction(qc3)
stateVec3 = state3.__array__()
print(stateVec3)

[0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j
 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j]


In [12]:
#QFT with 3 qubits

N = 8
A = 1/(np.sqrt(2))
omega_N = cmath.exp(2*math.pi*1j/N)
qft3 = BayesianNetwork([('q0m0','q0m1'), ('q0m0','q1m1'), ('q0m0','q2m1'), ('q1m0','q0m1'), ('q1m0','q1m1'), ('q1m0','q2m1'), ('q2m0','q0m1'), ('q2m0','q1m1'), ('q2m0','q2m1')])
cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[1], [0]])
cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[1], [0]])
cpd_q2m0 = TabularCPD(variable = 'q2m0', variable_card = 2, values = [[1], [0]])
cpd_q0m1 = TabularCPD(variable='q0m1', variable_card = 2, values = [[A,A,A,A,A,A,A,A], [A,A*(omega_N**4),A*(omega_N**8),A*(omega_N**12),A*(omega_N**16),A*(omega_N**20),A*(omega_N**24),A*(omega_N**28)]], evidence = ['q0m0','q1m0','q2m0'], evidence_card = [2,2,2])
cpd_q1m1 = TabularCPD(variable='q1m1', variable_card = 2, values = [[A,A,A,A,A,A,A,A], [A,A*(omega_N**2),A*(omega_N**4),A*(omega_N**6),A*(omega_N**8),A*(omega_N**10),A*(omega_N**12),A*(omega_N**14)]], evidence = ['q0m0','q1m0','q2m0'], evidence_card = [2,2,2])
cpd_q2m1 = TabularCPD(variable='q2m1', variable_card = 2, values = [[A,A,A,A,A,A,A,A], [A,A*(omega_N**1),A*(omega_N**2),A*(omega_N**3),A*(omega_N**4),A*(omega_N**5),A*(omega_N**6),A*(omega_N**7)]], evidence = ['q0m0','q1m0','q2m0'], evidence_card = [2,2,2])

"""
Let w = e^(i*pi/3)
U_QFT =
1/(2*sqrt(2))*
[1 1   1    1    1    1    1    1   ] 
[1 w   w^2  w^3  w^4  w^5  w^6  w^7 ]
[1 w^2 w^4  w^6  w^8  w^10 w^12 w^14]
[1 w^3 w^6  w^9  w^12 w^15 w^18 w^21]
[1 w^4 w^8  w^12 w^16 w^20 w^24 w^28]
[1 w^5 w^10 w^15 w^20 w^25 w^30 w^35]
[1 w^6 w^12 w^18 w^24 w^30 w^36 w^42]
[1 w^7 w^14 w^21 w^28 w^35 w^42 w^49]
"""

qft3.add_cpds(cpd_q0m0,cpd_q0m1,cpd_q1m0,cpd_q1m1,cpd_q2m0,cpd_q2m1)
qftInfer3 = VariableElimination(qft3)
q3 = qftInfer3.query(['q0m1','q1m1','q2m1'])
print(q3)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+---------+---------+---------+-----------------------+
| q0m1    | q2m1    | q1m1    |   phi(q0m1,q2m1,q1m1) |
+=========+=========+=========+=======================+
| q0m1(0) | q2m1(0) | q1m1(0) |        0.3536+0.0000j |
+---------+---------+---------+-----------------------+
| q0m1(0) | q2m1(0) | q1m1(1) |        0.3536+0.0000j |
+---------+---------+---------+-----------------------+
| q0m1(0) | q2m1(1) | q1m1(0) |        0.3536+0.0000j |
+---------+---------+---------+-----------------------+
| q0m1(0) | q2m1(1) | q1m1(1) |        0.3536+0.0000j |
+---------+---------+---------+-----------------------+
| q0m1(1) | q2m1(0) | q1m1(0) |        0.3536+0.0000j |
+---------+---------+---------+-----------------------+
| q0m1(1) | q2m1(0) | q1m1(1) |        0.3536+0.0000j |
+---------+---------+---------+-----------------------+
| q0m1(1) | q2m1(1) | q1m1(0) |        0.3536+0.0000j |
+---------+---------+---------+-----------------------+
| q0m1(1) | q2m1(1) | q1m1(1) |        0.3536+0.

In [13]:
# Obtain the ordering of the variables in the display above, as well as their values
q3Vars = q3.variables
q3Values = q3.values

print(q3Vars)
print(q3Values)

['q0m1', 'q2m1', 'q1m1']
[[[0.35355339+0.j 0.35355339+0.j]
  [0.35355339+0.j 0.35355339+0.j]]

 [[0.35355339+0.j 0.35355339+0.j]
  [0.35355339+0.j 0.35355339+0.j]]]


In [14]:
def bitListBack(n):
    N = 2**n
    numList = []
    numFormat = "0" + str(n) + "b"
    for i in range(N):
        numList.append((str(format(i,numFormat))[::-1]))
    return numList

def QiskitDict(stateVec,n):
    qbits = bitListBack(n)
    QbitDict = {}
    for i in range(2**n):
        QbitDict[qbits[i]]=np.round(stateVec[i],4)
    return QbitDict

print("1 qubit qft")
print(QiskitDict(stateVec1,1))

1 qubit qft
{'0': (0.7071+0j), '1': (0.7071+0j)}


In [15]:
print("2 qubit qft")
print(QiskitDict(stateVec2,2))

2 qubit qft
{'00': (0.5+0j), '10': (0.5+0j), '01': (0.5+0j), '11': (0.5+0j)}


In [16]:
print("3 qubit qft")
print(QiskitDict(stateVec3,3))

3 qubit qft
{'000': (0.3536+0j), '100': (0.3536+0j), '010': (0.3536+0j), '110': (0.3536+0j), '001': (0.3536+0j), '101': (0.3536+0j), '011': (0.3536+0j), '111': (0.3536+0j)}


In [17]:
# Obtain the ordering of the variables in the display above, as well as their values
valArr = q3.variables
valuesArr = q3.values

def create_var_order(orderArr):
    currNum = 0
    numArr = []
    for order in orderArr:
        if len(order) == 4:
            currNum = order[1]
        numArr.append(currNum)
    return numArr

def bitList(n):
    N = 2**n
    numList = []
    numFormat = "0" + str(n) + "b"
    for i in range(N):
        numList.append((str(format(i,numFormat))))
    return numList


def columnize(listOfBits):
    n = len(listOfBits[0])
    holder = []
    for i in range(n):
        col = []
        for bit in listOfBits:
            col.append(bit[i])
        holder.append(col)
    return holder

def reform():
    varOrderArr = create_var_order(valArr)
    listOfBits = bitList(len(varOrderArr))
    columns = columnize(listOfBits)
    rearrangedColumns = [None]*len(columns)
    for index, order in enumerate(varOrderArr):
        rearrangedColumns[index] = columns[int(order)]
    numOfCols = len(rearrangedColumns)
    bitStr = ""
    finalBitArr = []
    for bitIndex in range(len(rearrangedColumns[0])):
        for num in range(numOfCols):
            bitStr+=str(rearrangedColumns[num][bitIndex])
        finalBitArr.append(bitStr)
        bitStr = ""
    return finalBitArr

def createHashTable():
    resHash = {}
    bitOrder=reform()
    valuesFlat = valuesArr.flatten()
    for index, key in enumerate(bitOrder):
        resHash[key] = np.round(valuesFlat[index], 4)
    return resHash

PgmpyHash = createHashTable()

print(PgmpyHash == QiskitDict(stateVec3,3))
print(PgmpyHash)
print(QiskitDict(stateVec3,3))

True
{'000': (0.3536+0j), '010': (0.3536+0j), '001': (0.3536+0j), '011': (0.3536+0j), '100': (0.3536+0j), '110': (0.3536+0j), '101': (0.3536+0j), '111': (0.3536+0j)}
{'000': (0.3536+0j), '100': (0.3536+0j), '010': (0.3536+0j), '110': (0.3536+0j), '001': (0.3536+0j), '101': (0.3536+0j), '011': (0.3536+0j), '111': (0.3536+0j)}


In [18]:
"""
TO DO LIST:

1. Implement a function that automates the comparison between pgmpy and qiskit
2. Organize files to look nice
3. Push to Github

SOON:
4. Implement density matrices into pgmpy - eventually incorporate error events/Soham's work
5. Implement a function that generates the CPDs for hadamard, qft, pauli matrix gates, etc... save time

"""

"\nTO DO LIST:\n\n1. Implement a function that automates the comparison between pgmpy and qiskit\n2. Organize files to look nice\n3. Push to Github\n\nSOON:\n4. Implement density matrices into pgmpy - eventually incorporate error events/Soham's work\n5. Implement a function that generates the CPDs for hadamard, qft, pauli matrix gates, etc... save time\n\n"